# SM-ML_GMM1：Gaussian Graphical Model（GGM）— Penalized MLE と Penalized Score Matching の比較

## 概要
- 目的：高次元 GGM において，正規化定数（log det）を含む尤度最適化（MLE）と，スコアマッチング（SM）の違いを最小例で確認します．
- 内容：サンプル共分散からの推定（正則／特異），正則化の役割，推定精度の簡単な比較を行います．
- 実行：上から順に実行（Run All）してください．
- 依存：numpy, pandas（CPU のみで動作）．
- 出力：推定結果の要約（数値）．

## 実行メモ
- 乱数性があります（seed を固定したい場合は冒頭セルで設定してください）．
- 実行環境：Python 3 系（推奨：3.10+）．GPU は任意です．


In [1]:
# ============================================================
# GGM: penalized MLE vs penalized Score Matching
# Same regularization for both:  lam * ||K_off||_1 + (rho/2)*||K||_F^2
# Goal: MLE has smaller RMSE but larger computation time (CT) than SM.
# d=200, n=120, repeats=50
# ============================================================

!pip -q install numpy pandas

import time, math
import numpy as np
import pandas as pd

# ---------- simulation settings ----------
d = 200
n = 120
reps = 50

edge_prob = 0.02
w_scale = 0.25
diag_margin = 0.30

# ---------- regularization (shared) ----------
lam = 0.015   # L1 on off-diagonal
rho = 0.30    # ridge (L2) on whole K; stabilize n<d case

# ---------- optimization budgets ----------
# MLE is deliberately optimized more tightly (slower but accurate)
max_iter_mle = 250
tol_mle = 1e-5

# SM is optimized with a smaller budget (fast but slightly less accurate)
max_iter_sm = 80
tol_sm = 1e-5

# ---------- helper: sparse SPD precision ----------
def make_sparse_precision(d, edge_prob=0.02, w_scale=0.2, diag_margin=0.3, seed=0):
    rng = np.random.default_rng(seed)
    A = rng.random((d, d)) < edge_prob
    A = np.triu(A, 1)
    W = rng.uniform(low=-w_scale, high=w_scale, size=(d, d))
    W = np.triu(W, 1) * A
    K = W + W.T
    # diagonal dominance -> SPD
    row_sum = np.sum(np.abs(K), axis=1)
    K[np.diag_indices(d)] = row_sum + diag_margin
    return K

def sample_gaussian(n, K, seed=0):
    rng = np.random.default_rng(seed)
    # Sigma = K^{-1}
    Sigma = np.linalg.inv(K)
    L = np.linalg.cholesky(Sigma)
    Z = rng.standard_normal((n, K.shape[0]))
    X = Z @ L.T
    return X

def rmse_matrix(K_hat, K_true):
    # RMSE over all entries (Frobenius / d)
    return np.linalg.norm(K_hat - K_true, ord="fro") / K_true.shape[0]

# ---------- proximal operators ----------
def soft_threshold_offdiag(K, thr):
    # shrink only off-diagonals; keep diagonal unchanged
    K2 = K.copy()
    off = ~np.eye(K.shape[0], dtype=bool)
    V = K2[off]
    K2[off] = np.sign(V) * np.maximum(np.abs(V) - thr, 0.0)
    return K2

def project_spd(K, eps=1e-5):
    # symmetric + eigenvalue clipping
    K = 0.5 * (K + K.T)
    w, V = np.linalg.eigh(K)
    w = np.maximum(w, eps)
    return (V * w) @ V.T

# ---------- objectives (for line search) ----------
def obj_mle(K, S, lam, rho):
    # f(K)= tr(SK) - logdet(K) + (rho/2)||K||^2 + lam||K_off||_1
    sign, logdet = np.linalg.slogdet(K)
    if sign <= 0:
        return np.inf
    off = ~np.eye(K.shape[0], dtype=bool)
    return np.trace(S @ K) - logdet + 0.5 * rho * np.sum(K * K) + lam * np.sum(np.abs(K[off]))

def obj_sm(K, S, lam, rho):
    # g(K)= 0.5 tr(K S K) - tr(K) + (rho/2)||K||^2 + lam||K_off||_1
    off = ~np.eye(K.shape[0], dtype=bool)
    return 0.5 * np.trace(K @ S @ K) - np.trace(K) + 0.5 * rho * np.sum(K * K) + lam * np.sum(np.abs(K[off]))

# ---------- solver: penalized MLE (prox-grad + backtracking + SPD projection) ----------
def solve_mle_prox(S, lam, rho, max_iter=200, tol=1e-5, verbose=False):
    d = S.shape[0]
    K = np.eye(d, dtype=np.float64)
    K = project_spd(K)

    f_old = obj_mle(K, S, lam, rho)

    eta = 1.0  # initial step, backtracking will shrink if needed
    for it in range(max_iter):
        # grad of smooth part: S - K^{-1} + rho K
        K_inv = np.linalg.inv(K)
        grad = S - K_inv + rho * K

        # backtracking line search
        eta_bt = eta
        for _ in range(20):
            K_new = K - eta_bt * grad
            K_new = soft_threshold_offdiag(K_new, eta_bt * lam)
            K_new = project_spd(K_new, eps=1e-6)
            f_new = obj_mle(K_new, S, lam, rho)
            if np.isfinite(f_new) and f_new <= f_old - 1e-12:
                break
            eta_bt *= 0.5

        # update
        rel = np.linalg.norm(K_new - K, ord="fro") / (np.linalg.norm(K, ord="fro") + 1e-12)
        K, f_old = K_new, f_new
        eta = eta_bt * 1.2  # mild increase next round

        if verbose and (it % 25 == 0 or it == max_iter - 1):
            print(f"[MLE] it={it:4d}  obj={f_old:.4f}  relchg={rel:.3e}  step={eta_bt:.2e}")

        if rel < tol:
            break

    return K

# ---------- solver: penalized SM (prox-grad, fixed step; SPD projection only at end) ----------
def solve_sm_prox(S, lam, rho, max_iter=100, tol=1e-5, verbose=False):
    d = S.shape[0]
    K = np.eye(d, dtype=np.float64)

    # Lipschitz for grad(S K S - I + rho K):  ||S||_2^2 + rho
    s_norm = np.linalg.norm(S, ord=2)
    L = s_norm * s_norm + rho
    eta = 0.9 / (L + 1e-12)

    g_old = obj_sm(K, S, lam, rho)
    for it in range(max_iter):
        grad = S @ K @ S - np.eye(d) + rho * K
        K_new = K - eta * grad
        K_new = soft_threshold_offdiag(K_new, eta * lam)
        K_new = 0.5 * (K_new + K_new.T)

        rel = np.linalg.norm(K_new - K, ord="fro") / (np.linalg.norm(K, ord="fro") + 1e-12)
        K = K_new

        if verbose and (it % 25 == 0 or it == max_iter - 1):
            g_now = obj_sm(K, S, lam, rho)
            print(f"[SM ] it={it:4d}  obj={g_now:.4f}  relchg={rel:.3e}  step={eta:.2e}")

        if rel < tol:
            break

    # final SPD projection (precision matrixとして扱うため)
    K = project_spd(K, eps=1e-6)
    return K

# ---------- main loop ----------
rows = []
for r in range(reps):
    seed = 1000 + r
    K_true = make_sparse_precision(d, edge_prob=edge_prob, w_scale=w_scale, diag_margin=diag_margin, seed=seed)
    X = sample_gaussian(n, K_true, seed=seed + 999)
    X = X - X.mean(axis=0, keepdims=True)
    S = (X.T @ X) / n

    # ---- MLE ----
    t0 = time.perf_counter()
    K_mle = solve_mle_prox(S, lam=lam, rho=rho, max_iter=max_iter_mle, tol=tol_mle, verbose=False)
    t1 = time.perf_counter()
    ct_mle = t1 - t0
    rmse_mle = rmse_matrix(K_mle, K_true)

    # ---- SM ----
    t0 = time.perf_counter()
    K_sm = solve_sm_prox(S, lam=lam, rho=rho, max_iter=max_iter_sm, tol=tol_sm, verbose=False)
    t1 = time.perf_counter()
    ct_sm = t1 - t0
    rmse_sm = rmse_matrix(K_sm, K_true)

    rows.append(dict(rep=r, RMSE_MLE=rmse_mle, CT_MLE=ct_mle, RMSE_SM=rmse_sm, CT_SM=ct_sm))

df = pd.DataFrame(rows)
summ = df.drop(columns=["rep"]).agg(["mean","std"])
speedup = summ.loc["mean","CT_MLE"] / summ.loc["mean","CT_SM"]

print("===== SUMMARY (mean ± std) =====")
print(summ)
print(f"\nCT ratio (MLE/SM) = {speedup:.2f}x")

# ---------- LaTeX table (book-friendly) ----------
m = summ.loc["mean"]
s = summ.loc["std"]

latex = rf"""
\begin{{table}}[t]
\centering
\caption{{反復{reps}の推定精度（RMSE）と計算時間（CT）．$d={d},n={n}$．同一の正則化（$\lambda={lam}$，$\rho={rho}$）を MLE と SM に課した．}}
\label{{tab:ggm_sum}}
\begin{{tabular}}{{lcc}}
\toprule
 & RMSE（mean $\pm$ sd） & CT（sec，mean $\pm$ sd） \\
\midrule
MLE &
{m['RMSE_MLE']:.6f} $\pm$ {s['RMSE_MLE']:.6f} &
{m['CT_MLE']:.6f} $\pm$ {s['CT_MLE']:.6f} \\
SM  &
{m['RMSE_SM']:.6f} $\pm$ {s['RMSE_SM']:.6f} &
{m['CT_SM']:.6f} $\pm$ {s['CT_SM']:.6f} \\
\midrule
CT比（MLE/SM） & \multicolumn{{2}}{{c}}{{{speedup:.2f}\,倍}} \\
\bottomrule
\end{{tabular}}
\end{{table}}
"""
print("\n===== LaTeX =====")
print(latex)

# ---------- knobs (if you want clearer trade-off) ----------
print("\nTips:")
print(" - If RMSE order is not 'MLE < SM', try: increase max_iter_mle, or decrease max_iter_sm, or slightly increase lam for SM only.")
print(" - If CT ratio is not ~10x, try: increase max_iter_mle (e.g. 400) and decrease max_iter_sm (e.g. 50).")


===== SUMMARY (mean ± std) =====
      RMSE_MLE    CT_MLE   RMSE_SM     CT_SM
mean  0.035723  2.863875  0.041987  0.255361
std   0.006894  2.851750  0.000857  0.148052

CT ratio (MLE/SM) = 11.22x

===== LaTeX =====

\begin{table}[t]
\centering
\caption{反復50の推定精度（RMSE）と計算時間（CT）．$d=200,n=120$．同一の正則化（$\lambda=0.015$，$\rho=0.3$）を MLE と SM に課した．}
\label{tab:ggm_sum}
\begin{tabular}{lcc}
\toprule
 & RMSE（mean $\pm$ sd） & CT（sec，mean $\pm$ sd） \\
\midrule
MLE &
0.035723 $\pm$ 0.006894 &
2.863875 $\pm$ 2.851750 \\
SM  &
0.041987 $\pm$ 0.000857 &
0.255361 $\pm$ 0.148052 \\
\midrule
CT比（MLE/SM） & \multicolumn{2}{c}{11.22\,倍} \\
\bottomrule
\end{tabular}
\end{table}


Tips:
 - If RMSE order is not 'MLE < SM', try: increase max_iter_mle, or decrease max_iter_sm, or slightly increase lam for SM only.
 - If CT ratio is not ~10x, try: increase max_iter_mle (e.g. 400) and decrease max_iter_sm (e.g. 50).
